In [21]:
#Phishing URLs
import pandas as pd

phishing = pd.read_csv('DataSets/phishing.csv')
phish = phishing.sample(n=2000, replace=False)
phish.shape

(2000, 2)

In [22]:
#Select 75,000 legitimate URLs to be used (will shrink down once offline URLs are removed)
legitimate = pd.read_csv('DataSets/legitimate.csv')
legit = legitimate.sample(n=2000, replace=False)
legit.shape

(2000, 2)

In [23]:
#Add https:// to legit URL if not already present
legit["url"] = legit["url"].apply(lambda x: "https://" + x if not x.startswith("https://") and not x.startswith("http://") else x)
legit.head()

,url,label
70049,https://wpbeaverbuilder.com,0
192938,https://letgodbetrue.com,0
125346,https://adline.com,0
551673,https://mydinasty.com,0
680513,https://fenoxendustriyel.com,0


In [24]:
#Combine dataframe and shuffle data
df = pd.concat([phish, legit], ignore_index=True).sample(frac=1).reset_index(drop=True)
df.shape

(4000, 2)

In [25]:
import tldextract
import socket
from concurrent.futures import ThreadPoolExecutor

In [31]:
def is_valid_domain(domain):
    extracted = tldextract.extract(domain)
    return bool(extracted.domain) and bool(extracted.suffix)

def resolve_domain(domain):
    try:
        ip_address = socket.gethostbyname(domain)
        return ip_address
    except socket.gaierror:
        return None

def connect_to_socket(domain, port, timeout=5, max_retries=3):
    if not is_valid_domain(domain):
        print(f"Invalid domain: {domain}")
        return

    ip_address = resolve_domain(domain)
    if ip_address is None:
        print(f"Unable to resolve domain: {domain}")
        return

    retries = 0
    while retries < max_retries:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            sock.connect((ip_address, port))
            # Add your code to interact with the socket here
            sock.close()
            break
        except socket.timeout:
            print(f"Connection to {domain}:{port} timed out. Retrying...")
            retries += 1
        except socket.error as e:
            print(f"Error trying to connect to socket: {e}")
            retries += 1

In [32]:
def connect_to_socket(domain, port):
    if not is_valid_domain(domain):
        print(f"Invalid domain: {domain}")
        return

    ip_address = resolve_domain(domain)
    if ip_address is None:
        print(f"Unable to resolve domain: {domain}")
        return

    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((ip_address, port))
        # Add your code to interact with the socket here
        sock.close()
    except socket.error as e:
        print(f"Error trying to connect to socket: {e}")


In [33]:
def process_and_filter_urls(url, port=80):
    try:
        domain = tldextract.extract(url).domain + '.' + tldextract.extract(url).suffix
        if not is_valid_domain(domain):
            return False

        ip_address = resolve_domain(domain)
        if ip_address is None:
            return False

        # Attempt to connect to the socket without retries or timeout
        connect_to_socket(domain, port)

        return True

    except Exception as e:
        print(f"Error processing {url}: {e}")
        return False

In [34]:
def process_and_filter_urls_wrapper(args):
    index, url = args
    is_valid = process_and_filter_urls(url)
    return index, is_valid

In [35]:
def process_urls_multithreaded(df, num_threads=5):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Create a list of tuples containing the index and the URL
        input_tuples = [(index, row.url) for index, row in df.iterrows()]

        # Run the processing function in parallel
        results = list(executor.map(process_and_filter_urls_wrapper, input_tuples))

    valid_urls = [result[0] for result in results if result[1]]
    return df.loc[valid_urls]

In [36]:
filtered_df = process_urls_multithreaded(df, num_threads=5)

Error trying to connect to socket: [Errno 111] Connection refused
Error trying to connect to socket: [Errno 110] Connection timed out



KeyboardInterrupt

